In [1]:
import os

import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.vgg16 import preprocess_input


2024-02-07 15:06:06.364663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 15:06:06.364781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 15:06:06.541229: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# os.listdir('/kaggle/input')


In [ ]:
images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train",
                                      seed=1234, subset="both", validation_split=0.2, batch_size=32,
                                      labels="inferred", label_mode = "categorical")



### Here to note is the Input object and the RandomFlip layer

In [4]:
def load_model():
    # base_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    base_model = EfficientNetV2S(
        include_top=False,
        weights="imagenet",
        classes=120,
        classifier_activation="softmax",
        include_preprocessing=True,
    )

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(256, 256, 3)),
        tf.keras.layers.RandomFlip(mode='horizontal', seed=None),
        base_model
    ])

    return model


In [ ]:

def set_nontrainable_layers(model):
    model.trainable = False
    return model


In [5]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')
    model = tf.keras.Sequential([
      model,
      flattening_layer,
      dense_layer,
      prediction_layer
    ])

    return model


### Here for some reason I needed to add a .build() step with the parameter input_shape

In [8]:
def build_model():
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    model = add_last_layers(base_model)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])
    model.build(input_shape=(None,256,256,3))
    return model


### No separate preprocessing step for EfficientNet

In [9]:
# images_preprocessed = images[0].map(
#   lambda x, y: (preprocess_input(x), y))
# images_val_preprocessed = images[1].map(
#   lambda x, y: (preprocess_input(x), y))


In [ ]:
model = build_model()
model.summary()
es = EarlyStopping(patience=3)

# VGG16
# history = model.fit(images_preprocessed,
#          epochs=100,
#          batch_size=32,
#          callbacks=es,
#          validation_data=images_val_preprocessed,
#          verbose=1)

# EfficientNet
history = model.fit(images[0],
          epochs=100,
          batch_size=32,
          callbacks=es,
          validation_data=images[1],
          verbose=1)


In [ ]:
test_images = image_dataset_from_directory("/kaggle/input/cropped/test",
                                      seed=1234, batch_size=32,
                                      labels="inferred", label_mode = "categorical")
results = model.evaluate(test_images)


In [ ]:
# model.predict('/kaggle/input/testing/zeca.jpg')
